## Feature Engineering

## 모듈 임포트

In [1]:
import re
from textblob import TextBlob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import StandardScaler

In [27]:
import seaborn as sns
import matplotlib.pyplot as plt

## 데이터 임포트

In [7]:
train_df = pd.read_csv('../dataset/train.csv')
test_df = pd.read_csv('../dataset/test.csv')
submission_df = pd.read_csv('../dataset/sample_submission.csv')

In [ ]:
significant_features = [
    'len_diff', 'punc_diff',
    'sent_count_diff', 
    'lexical_div_diff',
    'repetition_diff', 
    # 'subjectivity_diff',
    'comma_ratio_diff', 
    # 'avg_word_len_diff'
]

target_words = [
    'company', 'brace', 'knee', 'progression', 
    'apologize', 'sorry'
]

In [9]:
def get_features(df):
    df['len_a'] = df['response_a'].str.len()
    df['len_b'] = df['response_b'].str.len()
    df['punc_a'] = df['response_a'].apply(lambda x: len(re.findall(r'[!?,;:]', str(x))))
    df['punc_b'] = df['response_b'].apply(lambda x: len(re.findall(r'[!?,;:]', str(x))))
    df['sent_a'] = df['response_a'].apply(lambda x: len(re.findall(r'[.!?]', str(x))))
    df['sent_b'] = df['response_b'].apply(lambda x: len(re.findall(r'[.!?]', str(x))))

    def ling_feat(text):
        words = re.findall(r'\b\w+\b', str(text).lower())
        uniq = set(words)
        lex_div = len(uniq)/(len(words)+1e-9)
        repetition = 1 - len(uniq)/(len(words)+1e-9)
        blob = TextBlob(str(text))
        subj = blob.sentiment.subjectivity
        return lex_div, repetition, subj
    
    for side in ['a', 'b']:
        df[[f'lex_{side}', f'rep_{side}', f'subj_{side}']] = df[f'response_{side}'].apply(
            lambda x: pd.Series(ling_feat(x))
        )
    
    df['comma_a'] = df['response_a'].apply(lambda x: len(re.findall(r'[;,]', str(x))) / (len(x.split()) + 1e-9))
    df['comma_b'] = df['response_b'].apply(lambda x: len(re.findall(r'[;,]', str(x))) / (len(x.split()) + 1e-9))
    # df['avglen_a'] = df['response_a'].apply(lambda x: np.mean([len(w) for w in str(x).split()]) if len(str(x).split()) > 0 else 0)
    # df['avglen_b'] = df['response_b'].apply(lambda x: np.mean([len(w) for w in str(x).split()]) if len(str(x).split()) > 0 else 0)

    # diff features
    df['len_diff'] = df['len_a'] - df['len_b']
    df['punc_diff'] = df['punc_a'] - df['punc_b']
    df['sent_count_diff'] = df['sent_a'] - df['sent_b']
    df['lexical_div_diff'] = df['lex_a'] - df['lex_b']
    df['repetition_diff'] = df['rep_a'] - df['rep_b']
    # df['subjectivity_diff'] = df['subj_a'] - df['subj_b']
    df['comma_ratio_diff'] = df['comma_a'] - df['comma_b']
    # df['avg_word_len_diff'] = df['avglen_a'] - df['avglen_b']
    
    # --- keyword presence ---
    text_cols = df[['prompt', 'response_a', 'response_b']].astype(str).agg(' '.join, axis=1)
    for word in target_words:
        df[f'contains_{word}'] = text_cols.str.contains(fr'\b{word}\b', case=False, na=False).astype(int)

    return df

In [10]:
train_df = get_features(train_df)
test_df = get_features(test_df)

In [11]:
train_df['label'] = np.select(
    [train_df['winner_model_a']==1, train_df['winner_model_b']==1, train_df['winner_tie']==1],
    [0,1,2]
)

In [20]:
keyword_features = [f'contains_{w}' for w in target_words]
all_features = significant_features + keyword_features

In [15]:
train_df

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,len_a,...,lexical_div_diff,repetition_diff,comma_ratio_diff,contains_company,contains_brace,contains_knee,contains_progression,contains_apologize,contains_sorry,label
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0,4538,...,-0.096081,0.096081,-0.022029,0,0,0,0,0,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0,3114,...,-0.124652,0.124652,-0.010712,0,0,0,0,0,0,1
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1,921,...,0.142640,-0.142640,-0.031418,0,0,0,0,0,0,2
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0,3182,...,-0.103679,0.103679,0.016538,0,0,0,0,0,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0,1300,...,-0.123966,0.123966,0.002994,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57472,4294656694,gpt-4-0613,claude-1,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",1,0,0,396,...,0.183474,-0.183474,0.073024,0,0,0,0,0,0,0
57473,4294692063,claude-2.0,llama-2-13b-chat,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba...",1,0,0,1707,...,0.041789,-0.041789,0.018849,0,0,0,0,0,0,0
57474,4294710549,claude-1,alpaca-13b,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us...",1,0,0,8683,...,-0.069617,0.069617,-0.017390,0,0,0,0,0,0,0
57475,4294899228,palm-2,tulu-2-dpo-70b,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or...",0,1,0,435,...,0.001637,-0.001637,0.010063,0,0,0,0,0,0,1


In [21]:
X = train_df[all_features]
y = train_df['label']

In [24]:
X.describe()

,len_diff,punc_diff,sent_count_diff,lexical_div_diff,repetition_diff,comma_ratio_diff,contains_company,contains_brace,contains_knee,contains_progression,contains_apologize,contains_sorry
count,57477.000000,57477.000000,57477.000000,57477.000000,57477.000000,57477.000000,57477.000000,57477.000000,57477.000000,57477.000000,57477.000000,57477.000000
mean,-8.343250,-0.093376,-0.103467,0.000207,-0.000207,-0.008809,0.047097,0.002888,0.003793,0.007760,0.044505,0.056057
std,1098.046783,19.128123,14.044901,0.175519,0.175519,1.317062,0.211848,0.053664,0.061470,0.087747,0.206215,0.230034
min,-52573.000000,-792.000000,-660.000000,-1.000000,-1.000000,-255.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-427.000000,-6.000000,-5.000000,-0.087695,-0.086965,-0.021958,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,416.000000,6.000000,4.000000,0.086965,0.087695,0.021247,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,43542.000000,1350.000000,545.000000,1.000000,1.000000,18.047619,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer

# 분석 및 혼합 스케일링(열별로 StandardScaler / RobustScaler / 그대로 유지)

# X, all_features 변수가 이미 존재한다고 가정
df = X.copy()

# 기본 통계량 & 이상치/분산 계산
summary = []
for col in df.columns:
  vals = df[col].dropna().astype(float)
  n_unique = df[col].nunique(dropna=True)
  is_binary = set(vals.unique()).issubset({0.0, 1.0}) or n_unique == 2
  var = vals.var()
  skew = vals.skew()
  q1, q3 = vals.quantile(0.25), vals.quantile(0.75)
  iqr = q3 - q1
  lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
  outlier_mask = (vals < lower) | (vals > upper)
  outlier_ratio = outlier_mask.sum() / max(len(vals), 1)
  low_variance = var < 1e-4  # 임계값은 데이터에 따라 조정 가능
  high_skew = abs(skew) > 1.0

  # 스케일러 권장: 이진 변수는 그대로, 이상치가 많거나 왜도가 큰 경우 Robust, 아니면 Standard
  if is_binary:
    recommended = 'passthrough (binary)'
  elif outlier_ratio > 0.05 or high_skew:
    recommended = 'RobustScaler'
  else:
    recommended = 'StandardScaler'

  summary.append({
    'feature': col,
    'dtype': str(df[col].dtype),
    'n_unique': n_unique,
    'is_binary': is_binary,
    'var': var,
    'skew': skew,
    'q1': q1,
    'q3': q3,
    'iqr': iqr,
    'outlier_ratio': outlier_ratio,
    'low_variance': low_variance,
    'recommended': recommended
  })

summary_df = pd.DataFrame(summary).set_index('feature')
print("Feature summary (first rows):")
print(summary_df[['dtype', 'is_binary', 'n_unique', 'var', 'skew', 'outlier_ratio', 'low_variance', 'recommended']].head(20))

# 그룹 분류
binary_cols = summary_df[summary_df['is_binary']].index.tolist()
robust_cols = summary_df[summary_df['recommended'] == 'RobustScaler'].index.difference(binary_cols).tolist()
std_cols = summary_df[summary_df['recommended'] == 'StandardScaler'].index.difference(binary_cols).tolist()

print("\nGroups:")
print("  binary_cols:", binary_cols)
print("  robust_cols:", robust_cols)
print("  std_cols:", std_cols)

# ColumnTransformer 구성 (빈 그룹은 생략)
transformers = []
if std_cols:
  transformers.append(('std', StandardScaler(), std_cols))
if robust_cols:
  transformers.append(('robust', RobustScaler(), robust_cols))
# passthrough for binary (유지)
if binary_cols:
  transformers.append(('passthrough_binary', 'passthrough', binary_cols))

if not transformers:
  raise RuntimeError("No features to transform. Check X/all_features.")

col_transformer = ColumnTransformer(transformers, remainder='drop', sparse_threshold=0)

# fit + transform -> DataFrame으로 복원
X_scaled_arr = col_transformer.fit_transform(df)
# ColumnTransformer 순서를 이용해 컬럼명 재구성
out_cols = []
for name, _, cols in transformers:
  if cols == 'passthrough' or name.startswith('passthrough'):
    # passthrough 사용한 경우, 실제 컬럼 순서가 입력 df에서 유지되므로 cols는 리스트로 받음
    out_cols.extend(cols if isinstance(cols, (list, tuple)) else list(cols))
  else:
    out_cols.extend(cols)
X_scaled_df = pd.DataFrame(X_scaled_arr, columns=out_cols, index=df.index)

# 결과 요약 출력
print("\nScaled features head:")
print(X_scaled_df.head())

print("\nScaled features summary (variance after scaling):")
print(X_scaled_df.var())

# 필요 시 파일/변수로 저장해 다음 셀에서 재사용 가능
# 예: X_scaled_df.to_csv('X_scaled_preview.csv', index=False)
# 또는 전역 변수로 보존
X_scaled_preview = X_scaled_df

Feature summary (first rows):
                        dtype  is_binary  n_unique           var        skew  \
feature                                                                        
len_diff                int64      False      6041  1.205707e+06   -1.124580   
punc_diff               int64      False       313  3.658851e+02    6.378323   
sent_count_diff         int64      False       255  1.972592e+02   -2.359540   
lexical_div_diff      float64      False     55950  3.080705e-02    0.025245   
repetition_diff       float64      False     55950  3.080705e-02   -0.025245   
comma_ratio_diff      float64      False     51816  1.734652e+00 -165.425305   
contains_company        int64       True         2  4.487974e-02    4.275875   
contains_brace          int64       True         2  2.879821e-03   18.527488   
contains_knee           int64       True         2  3.778502e-03   16.145383   
contains_progression    int64       True         2  7.699548e-03   11.219919   
contains_a

In [ ]:
# stratify로 클래스 비율 유지
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled_df, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"Train shape: {X_train.shape}, Validation shape: {X_val.shape}")
print(y.value_counts(normalize=True))

# --- 3. 베이스 모델 구성 ---
lr = LogisticRegression(max_iter=10000, multi_class='multinomial', solver='lbfgs', random_state=42)
rf = RandomForestClassifier(n_estimators=300, random_state=42)
gb = GradientBoostingClassifier(random_state=42)
lgb = LGBMClassifier(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# --- 4. Voting Ensemble ---
ensemble = VotingClassifier(
    estimators=[('lr', lr), ('rf', rf), ('gb', gb), ('lgb', lgb)],
    voting='soft'
)

# --- 5. 학습 ---
ensemble.fit(X_train, y_train)

# --- 6. 검증 ---
y_pred = ensemble.predict(X_val)
y_prob = ensemble.predict_proba(X_val)

acc = accuracy_score(y_val, y_pred)
loss = log_loss(y_val, y_prob)
print(f"\n✅ Validation Accuracy: {acc:.4f}")
print(f"✅ Validation LogLoss: {loss:.4f}")

# 추가 리포트
print("\nClassification Report:")
print(classification_report(y_val, y_pred, target_names=['A win', 'B win', 'Tie']))

# --- 7. 혼동 행렬 시각화 ---
plt.figure(figsize=(5,4))
sns.heatmap(confusion_matrix(y_val, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix (Validation)")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# --- 8. 최종 예측 ---
# test_df_scaled 준비
X_test_scaled = X_scaled_df.loc[test_df.index] if len(test_df) == len(X_scaled_df) else X_scaled_df.iloc[:len(test_df)]
test_pred = ensemble.predict_proba(X_test_scaled)

# --- 9. 제출 파일 생성 ---
submission_df['winner_model_a'] = test_pred[:, 0]
submission_df['winner_model_b'] = test_pred[:, 1]
submission_df['winner_tie'] = test_pred[:, 2]
submission_df.to_csv('submission_final_ensemble.csv', index=False)

print("\n🎯 submission_final_ensemble.csv saved successfully!")

# --- 10. Feature Importance (LGBM 기반) ---
importances = pd.Series(lgb.feature_importances_, index=X_scaled_df.columns)
plt.figure(figsize=(10,6))
importances.nlargest(15).plot(kind='barh')
plt.title("Top 15 Feature Importances (LightGBM)")
plt.gca().invert_yaxis()
plt.show()
